In [1]:
import glob
import json
from indra_world.ontology import load_world_ontology

In [2]:
new_onto_url = 'https://raw.githubusercontent.com/WorldModelers/Ontologies/'\
    '3.0/CompositionalOntology_metadata.yml'
old_onto_url = 'https://raw.githubusercontent.com/WorldModelers/Ontologies/'\
    '2.3/CompositionalOntology_metadata.yml'
new_onto = load_world_ontology(new_onto_url)
new_onto.initialize()
old_onto = load_world_ontology(old_onto_url)
old_onto.initialize()
new_onto_terms = {new_onto.get_id(node) for node in new_onto.nodes()}
old_onto_terms = {old_onto.get_id(node) for node in old_onto.nodes()}

INFO: [2021-09-21 16:40:57] indra_world.ontology.ontology - Initializing world ontology from https://raw.githubusercontent.com/WorldModelers/Ontologies/3.0/CompositionalOntology_metadata.yml
INFO: [2021-09-21 16:40:57] indra.ontology.ontology_graph - Building transitive closure for faster isa/partof lookups...
INFO: [2021-09-21 16:40:57] indra_world.ontology.ontology - Ontology has 580 nodes
INFO: [2021-09-21 16:40:57] indra_world.ontology.ontology - Initializing world ontology from https://raw.githubusercontent.com/WorldModelers/Ontologies/2.3/CompositionalOntology_metadata.yml
INFO: [2021-09-21 16:40:58] indra.ontology.ontology_graph - Building transitive closure for faster isa/partof lookups...
INFO: [2021-09-21 16:40:58] indra_world.ontology.ontology - Ontology has 534 nodes


In [3]:
print(f'Number of old ontology terms: {len(old_onto_terms)}')
print(f'Number of new ontology terms: {len(new_onto_terms)}')

Number of old ontology terms: 534
Number of new ontology terms: 580


In [4]:
for term in sorted(new_onto_terms-old_onto_terms):
    print(term)

wm/concept/agriculture/disease/poultry_disease
wm/concept/agriculture/poultry/backyard_poultry_farming
wm/concept/agriculture/poultry/day_old_chicks
wm/concept/agriculture/poultry/egg
wm/concept/agriculture/poultry/flock_size
wm/concept/civil_society
wm/concept/economy/unemployment
wm/concept/entity/border
wm/concept/entity/drone
wm/concept/entity/locations/neighboring_country
wm/concept/entity/military_exercise
wm/concept/entity/people/child_or_youth
wm/concept/entity/people/infant
wm/concept/entity/people/maternal
wm/concept/entity/people/migration/displaced_person
wm/concept/entity/people/migration/refugee
wm/concept/entity/people/militia
wm/concept/entity/people/young_adults
wm/concept/goods/agricultural/compound_feed
wm/concept/goods/agricultural/feed
wm/concept/health/protein_nutrient
wm/concept/inequality
wm/concept/information/intelligence_information
wm/concept/information/internet
wm/concept/information/social_media
wm/concept/infrastructure/voting_infrastructure
wm/concept/p

In [5]:
def get_grounding(concept):
    if 'theme' not in concept:
        return (concept['concept'], None, None, None)
    parts = ['theme', 'theme_property', 'process', 'process_property']
    return tuple(None if not concept[part] else concept[part] for part in parts)


def get_all_groundings(stmts):
    groundings = []
    for stmt in stmts:
        for concept in [stmt['obj'], stmt['subj']]:
            if 'theme' not in concept:
                continue
            groundings.append(get_grounding(concept))
    return groundings

def get_grounding_configs(unique_gr):
    grounding_configs = []
    for gr in unique_gr:
        parts = []
        for val, part in zip(gr, ['theme', 'property', 'process', 'process_property']):
            if val is not None:
                parts.append(part)
        grounding_configs.append('_'.join(parts))
    return grounding_configs

def get_all_grounding_terms(unique_gr):
    all_terms = set()
    for gr in unique_gr:
        for part in gr:
            if part:
                all_terms.add(part)
    return all_terms

# ATA

In [6]:
fnames = glob.glob('ATA/*.json')
all_groundings_ata = []
for fname in fnames:
    with open(fname, 'r') as fh:
        jj = json.load(fh)
        all_groundings_ata += get_all_groundings(jj['statements'])

In [7]:
print(f'All ATA statements: {int(len(all_groundings_ata)/2)}')

All ATA statements: 64


In [8]:
unique_gr_ata = set(all_groundings_ata)
print(f'Unique groundings in ATA models: {len(unique_gr_ata)}')

Unique groundings in ATA models: 28


In [9]:
Counter(get_grounding_configs(unique_gr_ata))

Counter({'theme_process': 14, 'theme': 11, 'theme_property': 3})

In [10]:
all_terms_ata = get_all_grounding_terms(unique_gr_ata)
nterms_ata = len(all_terms_ata)
nterms_ata_old = len(all_terms_ata & old_onto_terms)
nterms_ata_new = len(all_terms_ata & new_onto_terms)
nterms_ata_new_only = len((all_terms_ata & new_onto_terms) - (all_terms_ata & old_onto_terms))
nterms_ata_custom = len((all_terms_ata - new_onto_terms) - old_onto_terms)
print(f'All terms: {nterms_ata}, '
      f'Old terms: {nterms_ata_old}, '
      f'New terms: {nterms_ata_new}, '
      f'New only terms: {nterms_ata_new_only}, '
      f'Custom terms: {nterms_ata_custom}, ')

All terms: 28, Old terms: 21, New terms: 28, New only terms: 7, Custom terms: 0, 


In [11]:
for term in sorted((all_terms_ata & new_onto_terms) - (all_terms_ata & old_onto_terms)):
    print(term)

wm/concept/agriculture/poultry/egg
wm/concept/agriculture/poultry/flock_size
wm/concept/goods/agricultural/feed
wm/process/management
wm/process/production
wm/process/training/agriculture_training
wm/property/quality


In [12]:
for term in sorted((all_terms_ata - new_onto_terms) - old_onto_terms):
    print(term)

# NAF

In [29]:
fnames = glob.glob('NAF/*.json')

unique_stmts_naf = {}
for fname in fnames:
    if 'September Master' in fname:
        print(fname)
        with open(fname, 'r') as fh:
            jj = json.load(fh)
            for stmt in jj['statements']:
                if ('theme' not in stmt['obj']) and stmt['obj']['concept'].startswith('wm/'):
                    continue
                if ('theme' not in stmt['subj']) and stmt['subj']['concept'].startswith('wm/'):
                    continue
                unique_stmts_naf[(get_grounding(stmt['obj']),
                                  get_grounding(stmt['subj']))] = stmt
all_groundings_naf = []
for (sgr, ogr), stmt in unique_stmts_naf.items():
    all_groundings_naf += [sgr, ogr]

NAF/Alex September Master CAG.json
NAF/USAID September Master CAG.json
NAF/DoD September Master CAG.json
NAF/September Master CAG.json
NAF/State September Master CAG.json


In [30]:
print(f'All NAF statements: {len(unique_stmts_naf)}')

All NAF statements: 174


In [31]:
unique_gr_naf = set(all_groundings_naf)
print(f'Unique groundings in NAF models: {len(unique_gr_naf)}')

Unique groundings in NAF models: 103


In [32]:
Counter(get_grounding_configs(unique_gr_naf))

Counter({'theme_process': 24, 'theme': 74, 'theme_property': 5})

In [33]:
all_terms_naf = get_all_grounding_terms(unique_gr_naf)
nterms_naf = len(all_terms_naf)
nterms_naf_old = len(all_terms_naf & old_onto_terms)
nterms_naf_new = len(all_terms_naf & new_onto_terms)
nterms_naf_new_only = len((all_terms_naf & new_onto_terms) - (all_terms_naf & old_onto_terms))
nterms_naf_custom = len((all_terms_naf - new_onto_terms) - old_onto_terms)
print(f'All terms: {nterms_naf}, '
      f'Old terms: {nterms_naf_old}, '
      f'New terms: {nterms_naf_new}, '
      f'New only terms: {nterms_naf_new_only}, '
      f'Custom terms: {nterms_naf_custom}, ')

All terms: 108, Old terms: 76, New terms: 86, New only terms: 11, Custom terms: 21, 


In [34]:
for term in sorted((all_terms_naf & new_onto_terms) - (all_terms_naf & old_onto_terms)):
    print(term)

wm/concept/entity/military_exercise
wm/concept/entity/people/migration/refugee
wm/process/aid_or_assistance
wm/process/conflict/protest_or_demonstration
wm/process/imprisonment_or_detention
wm/process/legislation
wm/process/mitigation
wm/process/preparation
wm/process/production
wm/process/surveilance
wm/process/voting


In [35]:
for term in sorted((all_terms_naf - new_onto_terms) - old_onto_terms):
    print(term)

American Global Perception
Chinese Aid
Democracy Human Rights and Governance Aid
Environment Aid
Ethiopian Conflict
Ethiopian US Cooperation
Health Aid
Humanitarian Assistance Aid
International American Cooperation
Kenyan cooperation with the US
Peace and Security Aid Ethiopia
Peace and Security Aid Kenya
Peace and security Aid Somalia
Somali Conflict
Somali US Cooperation
Terrorist Attack
arrests of journalists
correction
education and social services aid
men_voting
personal data insecurity
